# Testing dynamicME on the latest cobrame model

In [ ]:
CONSTRAIN_O2 = True
TUNE_GLPK = True     # GLPK for glycerol
TUNE_RESPIRATION = True

In [ ]:
from dynamicme.dynamic import DynamicME

## 1. Build the ME model anew
### Only need to run this once or if a new version of the ME model is released

In [ ]:
# %run /source/ecolime/ecolime/build_me_model.py

## 2. Load the built ME model

In [ ]:
from cobrame.io.json import load_reduced_json_me_model, load_json_me_model
import pickle

In [ ]:
with open('/source/ecolime/ecolime/me_models/iJL1678b.pickle','rb') as f:
    me = pickle.load(f)

In [ ]:
rxn = me.reactions.get_by_id('GLYCtpp_REV_SPONT')
rxn.lower_bound = 0
rxn.upper_bound = 0

# 3. Set up the dynamic simulation

## Here, we will simulate the complex medium originally investigated by [Beg et al., 2007](http://www.pnas.org/content/104/31/12663)

### Metabolites to track

In [ ]:
tracked_mets = [
    me.metabolites.ac_p,
    me.metabolites.gal_p,
    me.metabolites.glc__D_p,
    me.metabolites.glyc_p,
    me.metabolites.lac__L_p, 
    me.metabolites.malt_p
    ]
extra_rxns_tracked = [me.reactions.biomass_dilution, me.reactions.EX_o2_e]

### Also track translation fluxes

In [ ]:
rxns_trsl = me.reactions.query('translation_')
print(len(rxns_trsl))
extra_rxns_tracked = extra_rxns_tracked + rxns_trsl

from cobrame.core.reaction import MetabolicReaction
import pandas as pd

rows_tracked = []

for met_track in tracked_mets:
    mid_c = met_track.id.replace('_p','_c')
    mid_e = met_track.id.replace('_p','_e')
    met_c = me.metabolites.get_by_id(mid_c)
    met_e = me.metabolites.get_by_id(mid_e)
    for rxn in met_track.reactions:
        if isinstance(rxn, MetabolicReaction) and rxn.keff and met_c in rxn.metabolites or met_e in rxn.metabolites:
            extra_rxns_tracked.append(rxn)
            rows_tracked.append({'met':met_track.id, 'rxn':rxn.id})
            #print met_track,'\t', rxn.id, '\t', rxn.keff          
            
df_tracked = pd.DataFrame(rows_tracked)

In [ ]:
dyme = DynamicME(me)

T = 10   # hours
V = 1.   # L
X0 = 0.00675 / V  # g/L
c0_dict = {'glc__D_e': 0.4,
           'lac__L_e': 0.4,
           'malt_e': 0.4,
           'gal_e': 0.4,
           'glyc_e':0.4,
           'ac_e': 0.0}
### Convert from g/L to mmol
for mid,c in c0_dict.items():
    met = me.metabolites.get_by_id(mid)
    c0_dict[met.id] = c / met.formula_weight * 1000

### Set max uptake rate
LB_EX = -10.
LB_O2 = -20.

lb_dict={}
ub_dict={}

for mid in c0_dict.keys():    
    rxn = dyme.get_exchange_rxn(mid)    
    if rxn.id == 'EX_o2_e':
        lb = LB_O2
    else:
        lb = LB_EX
    #rxn.lower_bound = lb
    lb_dict[rxn.id] = lb

#me.reactions.EX_o2_e.lower_bound = LB_O2
lb_dict[me.reactions.EX_o2_e.id] = LB_O2

print(c0_dict)
print(lb_dict)
print(ub_dict)

sim_params = {
    'T':T,
    'X0':X0,
    'c0_dict':c0_dict,
    'lb_dict':lb_dict,
    'ub_dict':ub_dict,
    'extra_rxns_tracked':extra_rxns_tracked,
    'ZERO_CONC':0.
}

# 4. Some manual keff tuning from literature (see Methods)

In [ ]:
growth_rxn = me.reactions.biomass_dilution
df_pert = df_tracked
groups = df_pert['met'].unique()

pert_rxns = [r for r in df_pert['rxn'].values]

group_rxn_dict = {met:df_pert[ df_pert['met']==met]['rxn'].values.tolist() for met in groups}
print(group_rxn_dict['malt_p'])
print(group_rxn_dict['glyc_p'])

In [ ]:
rows_perturb_manual = []

### Glycerol

### Glycerol kinase (GLPK) known to be rate-limiting (Applebee et al., 2011)

In [ ]:
### From Applebee et al. (2011)
vmax_glpk_ale = 86
vmax_glpk_wt  = 18.1

for rxn in me.process_data.GLYK.parent_reactions:
    extra_rxns_tracked.append(rxn)
    rows_perturb_manual.append({'met':'glyc_p', 'rxn':rxn.id})
    if TUNE_GLPK:
        keff1 = rxn.keff
        keff2 = keff1 * vmax_glpk_wt / vmax_glpk_ale
        print('Changing keff for %s from %g to %g'%(rxn.id, keff1, keff2))
        rxn.keff = keff2
        rxn.update()
        print(rxn.id, ':\t', rxn.reaction    )

### Respiration
Basan et al. (2015) Nat: proteome cost of energy biogenesis by respiration exceeds that by fermentation
- TCA
- Cytochromes
- ATP synthase

In [ ]:
### Isocitrate dehyrogenase keffs were way too high
for rxn in me.process_data.ICDHyr.parent_reactions:
    extra_rxns_tracked.append(rxn)
    rows_perturb_manual.append({'met':'o2_e', 'rxn':rxn.id})
    if TUNE_RESPIRATION:
        keff1 = rxn.keff
        keff2 = 11900./60   # BIONUMBERS http://bionumbers.hms.harvard.edu/bionumber.aspx?id=101152
        # 106.4    ### BRENDA for 40oC
        if keff2 < keff1:
            print('Changing keff for %s from %g to %g'%(rxn.id, keff1, keff2))
            rxn.keff = keff2
            rxn.update()
            print(rxn.id, ':\t', rxn.reaction)

### ATP Synthase: 
# F0F1: 270 +/- 40 s-1: http://www.ncbi.nlm.nih.gov/pubmed/9030757

### Cytochromes
# Zhuang (2011)
# Cyd-II: low cost
# Cyo: costly
# Cyd-I: preferred in microaerobic
# Cytochrome oxidase bo3 (ubiquinol-8: 4 protons)
for rxn in me.process_data.CYTBO3_4pp.parent_reactions:
    extra_rxns_tracked.append(rxn)
    rows_perturb_manual.append({'met':'o2_e', 'rxn':rxn.id})
    if TUNE_RESPIRATION:
        keff1 = rxn.keff
        keff2 = 300. # http://www.brenda-enzymes.org/enzyme.php?ecno=1.9.3.1
        if keff2 < keff1:
            print('Changing keff for %s from %g to %g'%(rxn.id, keff1, keff2))
            rxn.keff = keff2
            rxn.update()
            print(rxn.id, ':\t', rxn.reaction)
            
# Cytochrome oxidase bd (menaquinol-8: 2 protons)
for rxn in me.process_data.CYTBD2pp.parent_reactions:
    extra_rxns_tracked.append(rxn)
    rows_perturb_manual.append({'met':'o2_e', 'rxn':rxn.id})
    if TUNE_RESPIRATION:
        keff1 = rxn.keff
        keff2 = keff1 / 2.
        if keff2 < keff1:
            print('Changing keff for %s from %g to %g'%(rxn.id, keff1, keff2))
            rxn.keff = keff2
            rxn.update()
            print(rxn.id, ':\t', rxn.reaction)
            
# Cytochrome oxidase bd (ubiquinol-8: 2 protons)
for rxn in me.process_data.CYTBDpp.parent_reactions:
    extra_rxns_tracked.append(rxn)
    rows_perturb_manual.append({'met':'o2_e', 'rxn':rxn.id})
    if TUNE_RESPIRATION:
        keff1 = rxn.keff
        keff2 = keff1 / 2.
        if keff2 < keff1:
            print('Changing keff for %s from %g to %g'%(rxn.id, keff1, keff2))
            rxn.keff = keff2
            rxn.update()
            print(rxn.id, ':\t', rxn.reaction)

In [ ]:
df_pert_manual = pd.DataFrame(rows_perturb_manual).drop_duplicates()
print(df_pert_manual.shape)

In [ ]:
# Append timestamp to filename
import time
timestr = '_'+time.strftime("%Y%m%d_%H%M")
print(timestr)

# Append simulation conditions to filename
simstr = ''
if TUNE_GLPK: simstr = simstr + '_GLPK'
if TUNE_RESPIRATION: simstr = simstr + '_RESP'

print(simstr)

# 5. Perform DynamicME simulation

In [ ]:
dyme = DynamicME(me)

In [ ]:
import time

tic = time.time()

if CONSTRAIN_O2:
    result = dyme.simulate_batch(T, c0_dict, X0, prec_bs=1e-3, ZERO_CONC=0., extra_rxns_tracked=extra_rxns_tracked,
                             lb_dict={'EX_o2_e': -18.5}, verbosity=1)
else:
    result = dyme.simulate_batch(T, c0_dict, X0, prec_bs=1e-3, ZERO_CONC=0., extra_rxns_tracked=extra_rxns_tracked,
                             lb_dict={}, verbosity=0)
    
toc = time.time()-tic

In [ ]:
print('%g seconds'%toc)

# 6. Convert solution's metabolite concentrations from mM to g/L

In [ ]:
from dynamicme.dynamic import ParamOpt

popt = ParamOpt(me, sim_params, growth_rxn=growth_rxn.id)
sol = popt.compute_conc_profile(result)

In [ ]:
df_mw = pd.DataFrame([{'id':m.id,'mass':m.mass if hasattr(m,'mass') else m.formula_weight,'name':m.name} for m in me.metabolites])

sol_gL = sol.copy()
variables = c0_dict.keys()

for col in sol.columns:
    if col in variables:
        c_mM = sol[col]
        mw = df_mw[ df_mw.id==col].mass.values[0]
        c_gL = c_mM * mw / 1000.
        sol_gL[col] = c_gL
        
df_ex = pd.DataFrame(result['ex_flux'])

# 7. Plot metabolite concentration profiles

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

dsplot = sol_gL[[c for c in sol_gL.columns if c != 'biomass' and 'EX_' not in c]].melt(id_vars='time')

dsplot.variable.unique()

palette = ['#660066','#00FF00','#000000','#FF00FF','#0000FF','#FF0000']
sns.set_palette(palette)
g = sns.FacetGrid(dsplot, hue='variable', size=5)
g.map(plt.plot, 'time','value')
g.add_legend()